1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (input이 길면) 답변 생성이 오래 걸림
3. embedding -> vector database에 저장
4. 질문이 있을 때, vector database에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [ ]:
%pip install langchain_community docx2txt langchain_chroma langchainhub

In [12]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 소득세법
# document_name = './tax_with_markdown.docx'
# 민법
# document_name = './civil_with_markdown.docx'
# 형법
# document_name = './criminal_with_markdown.docx'
# 도로교통법
# document_name = './traffic_with_markdown.docx'
# 근로기준법
document_name = './labor_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [13]:
len(document_list)

29

In [14]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [15]:
from langchain_chroma import Chroma

is_create = True
# is_create = False
# 소득세법
# collection_name = 'chroma-tax'
# 민법
# collection_name = 'chroma-civil'
# 형법
# collection_name = 'chroma-criminal'
# 도로교통법
# collection_name = 'chroma-traffic'
# 근로기준법
collection_name = 'chroma-labor'

if is_create:
    # 데이터를 처음 저장할 때 
    database = Chroma.from_documents(
        documents=document_list, 
        embedding=embedding,
        collection_name=collection_name,
        persist_directory='./chroma'
    )
else:
    # 이미 저장된 데이터를 사용할 때 
    database = Chroma(
        collection_name=collection_name,
        persist_directory='./chroma',
        embedding_function=embedding
    )

In [16]:
# 소득세법
# query = '연봉 5천만원인 직장인의 소득세는?'
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
# query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'
# 도로교통법
# query = '음주운전 단속 기준은 어떻게 되나요?'
# 근로기준법
query = '임금 체불 시 어떻게 대응해야 하나요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [18]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [20]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [21]:
ai_message = qa_chain.invoke(query)

In [22]:
ai_message

{'query': '임금 체불 시 어떻게 대응해야 하나요?',
 'result': '임금 체불 시, 먼저 고용노동부에 신고하여 체불임금 진정을 접수할 수 있습니다. 또한, 민사집행법에 따른 지급명령이나 소송을 통해 임금채권을 확보할 수 있으며, 체불사업주의 명단 공개 등을 통해 압박할 수도 있습니다. 필요 시, 직상 수급인에게 연대책임을 요구하여 임금 지급을 청구할 수 있습니다.'}